In [60]:
# 원시 데이터 샘플링

import os
import shutil
from pathlib import Path

input_path_1 = './data_raw/train_label/TL_01.딸기_001.설향_01.정상'
input_path_2 = './data_raw/train_label/TL_01.딸기_001.설향_02.역병'
input_path_3 = './data_raw/train_label/TL_01.딸기_001.설향_03.시들음병'
input_path_4 = './data_raw/train_label/TL_01.딸기_001.설향_04.잎끝마름'
input_path_5 = './data_raw/train_label/TL_01.딸기_001.설향_05.황화'
output_path_1 = './data_raw/train_label/sampling_normal'
output_path_2 = './data_raw/train_label/sampling_blight'
output_path_3 = './data_raw/train_label/sampling_wilt'
output_path_4 = './data_raw/train_label/sampling_scorch'
output_path_5 = './data_raw/train_label/sampling_chlorosis'

input_paths = [input_path_1, input_path_2, input_path_3, input_path_4, input_path_5]
output_paths = [output_path_1, output_path_2, output_path_3, output_path_4, output_path_5]
num_files = 12000

def copy_files(input_path, output_path, num_files=12000):
    # output폴더를 여기서 생성하게 할 경우에는 아래의 Path 기능 사용
    # Path(output_path).mkdir(parents=True, exist_ok=True)
    
    files = [f for f in os.listdir(input_path) if f.endswith('.json')]
    files_to_copy = files[-num_files:]

    for file in files_to_copy:
        shutil.copy(os.path.join(input_path, file), os.path.join(output_path, file))

    return f"{len(files_to_copy)} files coppied from {input_path} to {output_path}"


In [61]:
copy_results = [copy_files(input_path, output_path, num_files) for input_path, output_path in zip(input_paths, output_paths)]
copy_results

KeyboardInterrupt: 

In [62]:
# 원시 데이터 데이터 프레임화(기존 작업 코드 재활용 후 여기서 현재 계획대로 데이터프레임 분리)
# 종류별로 작업 후 나중에 병합(concat, axis=0)

import json
import pandas as pd

def json_to_df(folder_path):
    df_list = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.json'):
            try:
                with open(file_path, 'r', encoding='utf-8') as file:
                    data = json.load(file)

                df_list.append(pd.json_normalize(data))
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON in file {filename}: {e}")

    if df_list:
        merged_df = pd.concat(df_list, ignore_index=True)
    else:
        print("No valid JSON files found")

    print('merged_df')
    return merged_df


In [63]:
folder_paths = [r'./data_raw/train_label/sampling_normal', r'./data_raw/train_label/sampling_blight',
                r'./data_raw/train_label/sampling_wilt', r'./data_raw/train_label/sampling_scorch',
                r'./data_raw/train_label/sampling_chlorosis']

merged_normal = json_to_df(folder_paths[0])
merged_blight = json_to_df(folder_paths[1])
merged_wilt = json_to_df(folder_paths[2])
merged_scorch = json_to_df(folder_paths[3])
merged_chlorosis = json_to_df(folder_paths[4])

merged_df
merged_df
merged_df
merged_df
merged_df


In [64]:
print(merged_normal.head(1))
print(merged_blight.head(1))
print(merged_wilt.head(1))
print(merged_scorch.head(1))
print(merged_chlorosis.head(1))

                                        environments  \
0  [{'farm_id': 'AI22_006', 'receive_date': '2022...   

                                         annotations  \
0  [{'coordinates': [256.48, 505.46, 401.62, 505....   

                                           licenses  \
0  [{'id': 1, 'name': 'CCMediaService', 'url': ''}]   

                                          categories  images.image_id  \
0  [{'id': 0, 'name': '잎', 'supercategory': 'none...           110703   

  images.farm_id images.crops_id images.crops images.kind_type  \
0       AI22_006    BERRY_55_008           딸기               설향   

             images.file_path  ... growth_index.fr2_cnt growth_index.fr3_cnt  \
0  /원천데이터/01.딸기/001.설향/01.정상/  ...                 None                 None   

  growth_index.fr_weight etc_infor.crops_id etc_infor.create_date  \
0                   None       BERRY_55_008                         

  etc_infor.inform images.stem images.grpoint images.fruit images.flower  
0       

In [68]:
merged_normal.to_csv('./data_raw/train_label/merged_df/merged_raw_normal.csv', encoding='utf-8', index=False)
merged_blight.to_csv('./data_raw/train_label/merged_df/merged_raw_blight.csv', encoding='utf-8', index=False)
merged_wilt.to_csv('./data_raw/train_label/merged_df/merged_raw_wilt.csv', encoding='utf-8', index=False)
merged_scorch.to_csv('./data_raw/train_label/merged_df/merged_raw_scorch.csv', encoding='utf-8', index=False)
merged_chlorosis.to_csv('./data_raw/train_label/merged_df/merged_raw_chlorosis.csv', encoding='utf-8', index=False)

In [69]:
# 데이터프레임 정제(df_main, df_bbox)
"""
raw 데이터프레임 컬럼 추출 요소
*environments: value_{ti, hi, ci, ir, sr, tl, ei, pl, cl, el, hl, pi, RP}
*annotations
*categories
*width
*height
*images.fname
*images.disease_class
*images.disease_cause_method
"""

import pandas as pd
import json

raw_normal = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_normal.csv', encoding='utf-8')
raw_blight = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_blight.csv', encoding='utf-8')
raw_wilt = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_wilt.csv', encoding='utf-8')
raw_scorch = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_scorch.csv', encoding='utf-8')
raw_chlorosis = pd.read_csv('./data_raw/train_label/merged_df/merged_raw_chlorosis.csv', encoding='utf-8')


In [70]:
raw_normal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   environments                 12000 non-null  object 
 1   annotations                  12000 non-null  object 
 2   licenses                     12000 non-null  object 
 3   categories                   12000 non-null  object 
 4   images.image_id              12000 non-null  int64  
 5   images.farm_id               12000 non-null  object 
 6   images.crops_id              12000 non-null  object 
 7   images.crops                 12000 non-null  object 
 8   images.kind_type             12000 non-null  object 
 9   images.file_path             12000 non-null  object 
 10  images.fname                 12000 non-null  object 
 11  images.fext                  12000 non-null  object 
 12  images.width                 12000 non-null  int64  
 13  images.height   

In [24]:
def extract_cols(df, col_list):
    df_sel = df[col_list].copy()
    df_sel['annotations'] = df_sel['annotations'].apply(lambda x: json.loads(x.replace("'", "\"")))
    df_sel['categories'] = df_sel['categories'].apply(lambda x: json.loads(x.replace("'", "\"")))
    df_sel['environments'] = df_sel['environments'].apply(lambda x: json.dumps(x))
    df_sel['environments'] = df_sel['environments'].apply(json.loads)

    max_len = df_sel.loc[df_sel['annotations'].str.len().idxmax()]
    max_len_val = max_len['annotations']
    data = max_len_val

    if data and isinstance(data, list):
        item = data[-1]
        cat_id = item['category_id']
        print(cat_id)
    else:
        print("error")

    length = cat_id + 1
    zero_box = [0,0,0,0]
    unknown_status = 'F'

    for i in range(length):
        df_sel[f'bbox_{i}'] = None
        df_sel[f'status_{i}'] = None
        df_sel[f'name_{i}'] = None

        df_sel[f'ti_value_{i}'] = None
        df_sel[f'hi_value_{i}'] = None
        df_sel[f'ci_value_{i}'] = None
        df_sel[f'ir_value_{i}'] = None
        df_sel[f'tl_value_{i}'] = None
        df_sel[f'ei_value_{i}'] = None
        df_sel[f'pl_value_{i}'] = None
        df_sel[f'sr_value_{i}'] = None
        df_sel[f'cl_value_{i}'] = None
        df_sel[f'el_value_{i}'] = None
        df_sel[f'hl_value_{i}'] = None
        df_sel[f'pi_value_{i}'] = None
        df_sel[f'rp_value_{i}'] = None


    for idx, row in df_sel.iterrows():
        annotation_data = row['annotations']
        for k in range(len(annotation_data)):
            if k < len(annotation_data):
                df_sel.at[idx, f'bbox_{k}'] = annotation_data[k]['bbox']
                df_sel.at[idx, f'status_{k}'] = annotation_data[k]['disease_status']
            else:
                df_sel.at[idx, f'bbox_{k}'] = zero_box
                df_sel.at[idx, f'status_{k}'] = unknown_status

    for idx, row in df_sel.iterrows():
        name_data = row['categories']
        for k in range(len(name_data)):
            if k < len(name_data):
                df_sel.at[idx, f'name_{k}'] = name_data[k]['name']
            else:
                df_sel.at[idx, f'name_{k}'] = 'unknown'

    for idx, row in df_sel.iterrows():
        value_data = row['environments']
        if isinstance(value_data, str):
            value_data = json.loads(value_data)
        for k in range(len(value_data)):
            if k < len(value_data):
                df_sel.at[idx, f'ti_value_{k}'] = value_data[k]['ti_value']
                df_sel.at[idx, f'hi_value_{k}'] = value_data[k]['hi_value']
                df_sel.at[idx, f'ci_value_{k}'] = value_data[k]['ci_value']
                df_sel.at[idx, f'ir_value_{k}'] = value_data[k]['ir_value']
                df_sel.at[idx, f'tl_value_{k}'] = value_data[k]['tl_value']
                df_sel.at[idx, f'ei_value_{k}'] = value_data[k]['ei_value']
                df_sel.at[idx, f'pl_value_{k}'] = value_data[k]['pl_value']
                df_sel.at[idx, f'sr_value_{k}'] = value_data[k]['sr_value']
                df_sel.at[idx, f'cl_value_{k}'] = value_data[k]['cl_value']
                df_sel.at[idx, f'el_value_{k}'] = value_data[k]['el_value']
                df_sel.at[idx, f'hl_value_{k}'] = value_data[k]['hl_value']
                df_sel.at[idx, f'pi_value_{k}'] = value_data[k]['pi_value']
                df_sel.at[idx, f'rp_value_{k}'] = value_data[k]['rp_value']
            else:
                df_sel.at[idx, f'ti_value_{k}'] = 'no_value'
                df_sel.at[idx, f'hi_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ci_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ir_value_{k}'] = 'no_value'
                df_sel.at[idx, f'tl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'ei_value_{k}'] = 'no_value'
                df_sel.at[idx, f'pl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'sr_value_{k}'] = 'no_value'
                df_sel.at[idx, f'cl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'el_value_{k}'] = 'no_value'
                df_sel.at[idx, f'hl_value_{k}'] = 'no_value'
                df_sel.at[idx, f'pi_value_{k}'] = 'no_value'
                df_sel.at[idx, f'rp_value_{k}'] = 'no_value'


    df_sel.drop(columns=['environments','annotations', 'categories'], axis=1, inplace=True)

    return df_sel



In [41]:
normal_sample = raw_normal[:5].copy()
normal_sample.head(1)

,environments,annotations,licenses,categories,images.image_id,images.farm_id,images.crops_id,images.crops,images.kind_type,images.file_path,...,growth_index.fr2_cnt,growth_index.fr3_cnt,growth_index.fr_weight,etc_infor.crops_id,etc_infor.create_date,etc_infor.inform,images.stem,images.grpoint,images.fruit,images.flower
0,"[{'farm_id': 'AI22_006', 'receive_date': '2022...","[{'coordinates': [256.48, 505.46, 401.62, 505....","[{'id': 1, 'name': 'CCMediaService', 'url': ''}]","[{'id': 0, 'name': '잎', 'supercategory': 'none...",110703,AI22_006,BERRY_55_008,딸기,설향,/원천데이터/01.딸기/001.설향/01.정상/,...,NaN,NaN,NaN,BERRY_55_008,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
# 단계별 함수 생성
    # environments 컬럼에서 각 value 값들의 평균 추출

import pandas as pd
import numpy as np
import json

def calculate_values(dicts_list):
    sums = {}
    counts = {}
    for d in dicts_list:
        for key, value in d.items():
            if key.endswith('_value'):
                if value is not None:
                    if key in sums:
                        sum[key] += float(value)
                        counts[key] += 1
                    else:
                        sum[key] = float(value)
                        counts[key] = 1
    averages = {key: sums[key] / counts[key] if key in counts else None for key in sums}
    return averages


json_str = normal_sample['environments'][0]
str_corrected = json_str.replace("None", "null").replace("'", "\"")
environment_json = json.loads(str_corrected)
averages = calculate_values(environment_json)

TypeError: 'builtin_function_or_method' object does not support item assignment

In [90]:
json_str = normal_sample['environments'][0]
str_corrected = json_str.replace("None", "null").replace("'", "\"")

env = json.loads(str_corrected)

print(type(env))
print(env)

<class 'list'>
[{'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:00:26', 'id': 0, 'ti_value': '16.7', 'hi_value': '93.4', 'ci_value': '595', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:01:23', 'id': 1, 'ti_value': '16.7', 'hi_value': '93.5', 'ci_value': '597', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006', 'receive_date': '2022-10-05 00:02:24', 'id': 2, 'ti_value': '16.7', 'hi_value': '93.5', 'ci_value': '597', 'ir_value': None, 'tl_value': None, 'ei_value': '0.07', 'pl_value': '6.2', 'sr_value': '0', 'cl_value': '20.2', 'el_value': '3.20', 'hl_value': '3.9', 'pi_value': '11.0', 'rp_value': None}, {'farm_id': 'AI22_006

In [94]:
print(type(env[0]['ti_value']))

<class 'str'>


In [89]:
"""
각 _value에 대한 빈 리스트 생성
각 딕셔너리에서 해당 값을 대상 빈 리스트에 append -> 각 값들이 집합한 리스트 생성
리스트의 요소 타입을 str -> float으로 변환(현재 딕셔너리에 저장된 각 숫자값들은 str로 되어있음)
리스트별로 요소의 값들의 평균을 계산(sum/len)
각 _value이름을 갖는 컬럼을 생성하고 거기에 값 입력

위의 과정을 데이터프레임 전체에 적용하는 함수 생성
"""

145


In [ ]:
"""
bbox, name, status 분리 작업

딕셔너리 집합을 풀어서 각각 row로 할당(공유 정보는 동일하게 컬럼에 데이터 할당하기: 예; images.fname)
각 row에서 bbox, name, status 분리하여 각 이름을 갖는 컬럼에 할당
id할당
5개 각각에 위의 작업 실행 후 최종적으로 main_df, info_df 분리하여 통합 데이터프레임 2개로 정리하기
"""

In [ ]:
# 라벨 데이터에 해당하는 원본 이미지 파일 복사

In [ ]:
# df_main, df_bbox 결과 확인